<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/A_1_x1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A.1.4. 데이터 분배 확인

 본 절에서는 [기존 코드](https://github.com/secutron/Practice_Ignite/blob/main/A_1.ipynb)를 수정하여, 학습 데이터가 분산처리를 위해 생성된 각 프로세스로 배분된 상황을 확인한다. 

In [ ]:
import os

gpu_gtg = False
if int(os.environ.get("COLAB_GPU")) > 0:
    gpu_gtg = "COLAB_GPU" in os.environ

tpu_gtg = "COLAB_TPU_ADDR" in os.environ

if tpu_gtg: # tpu
    print("TPU")
    #VERSION = "nightly"

    # https://github.com/pytorch/builder/pull/750
    VERSION = "20210304" # was 20200607" 

    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

TPU
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  69135      0 --:--:-- --:--:-- --:--:-- 69135
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20210304 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 1.5 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.278 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompa

In [ ]:
!pip install --pre pytorch-ignite

     |████████████████████████████████| 233 kB 5.5 MB/s 


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models

import torchsummary

import ignite
import ignite.distributed as idist
from ignite.engine import Engine, Events, create_supervised_evaluator, create_supervised_trainer
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.utils import setup_logger

병렬 컨텍스트에서 실행되는 training 함수 내부에 미니배치(minibatch)의 첫번 째 이터레이션(iteration)에 해당하는 샘플 이미지를 할당된 VM에 바로 저장하도록 코드를 추가하였다. 


In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE) # controls default text sizes
plt.rc('xtick', labelsize=SMALL_SIZE) # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE) # fontsize of the tick labels

def training(local_rank, config, **kwargs):
    print("local rank: ", local_rank)

    ###########################################################
    # 데이터 준비
    train_transform = transforms.Compose(
        [
            transforms.Pad(4),
            transforms.RandomCrop(32, fill=128),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

    #test_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])
    test_transform = transforms.Compose([transforms.ToTensor(),])

    if idist.get_local_rank() > 0:
        idist.barrier()

    trainset = torchvision.datasets.CIFAR10(root=config["data_path"], train=True, download=True, transform=train_transform)
    testset = torchvision.datasets.CIFAR10(root=config["data_path"], train=False, download=True, transform=test_transform)

    if idist.get_local_rank() == 0:
        idist.barrier()

    trainloader = idist.auto_dataloader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"], drop_last=True)
    testloader = idist.auto_dataloader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=config["num_workers"],)


    #########################################################
    # 데이터 분배 확인을 위한 코드 추가
    # 좌측 페인에서 파일 아이콘을 선택하면 보이는 리스트에서 이미지 확인 가능
    images, labels = next(iter(trainloader))
    # 64개
    h = 8
    w = 8

    fig, ax = plt.subplots(h, w, figsize=(3 * w, 3 * h))
    for n1 in range(h):
        for n2 in range(w):
            n12 = n1*w + n2
            ax[n1, n2].imshow(np.transpose(images[n12].cpu().numpy(), (1, 2, 0)))
            ax[n1, n2].set_title(np.round(labels[n12].cpu().numpy(), 1))
    plt.savefig(f"/content/batch_64_{idist.get_local_rank()}.png")




    ###########################################################
    # 모델, 옵티마이저, 로스, 트레이너, 이밸류에이터
    num_classes = 10
    model = models.resnet18(num_classes = num_classes)
       
    model = idist.auto_model(model)
    optimizer = idist.auto_optim(optim.Adam(model.parameters(), lr=0.001))

    criterion = nn.CrossEntropyLoss().to(idist.device())

    trainer = create_supervised_trainer(model, optimizer, criterion, device=idist.device())
    trainer.logger = setup_logger("hkim-trainer")

    metrics = {
        'accuracy':Accuracy(),
        'ce':Loss(criterion),
    }

    val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=idist.device())
    val_evaluator.logger = setup_logger("hkim-val_evaluator")

    # track a running average of the scalar loss output for each batch.
    RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

    ###########################################################
    # 이벤트

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        state = val_evaluator.run(testloader)
        metrics = val_evaluator.state.metrics
        accuracy = metrics['accuracy']*100
        loss = metrics['ce']
        log_metrics(val_evaluator.logger, state.epoch, state.times["COMPLETED"], "validation evaluator", state.metrics)

    trainer.run(trainloader, max_epochs=config["num_epochs"])    

In [ ]:
config = {
    "seed": 543,
    "data_path" : "./cifar10",
    "output_path" : "./output-cifar10/",
    "model" : "resnet18",
    "batch_size" : 512,
    "momentum" : 0.9,
    "weight_decay" : 1e-4,
    "num_workers" : 2,
    "num_epochs" : 5,
    "learning_rate" : 0.4,
    "num_warmup_epochs" : 4,
    "validate_every" : 3, 
    "checkpoint_every" : 1000,
    "backend" : None, 
    "resume_from" : None, 
    "log_every_iters" : 15,
    "nproc_per_node" : None, 
    "stop_iteration" : None, 
    "with_amp" : False,
    "log_interval" : 10,
    "verbose_set" : False,
    "verbose_set2" : False,
    "verbose_loader" : False

}

if not (tpu_gtg or gpu_gtg): # cpu
    config["backend"] = 'gloo'
    config["nproc_per_node"] = 8
elif gpu_gtg: # gpu
    config["backend"] = 'nccl'
    config["nproc_per_node"] = 1
elif tpu_gtg: # tpu
    config["backend"] = 'xla-tpu'
    config["nproc_per_node"] = 8
else: # error
    raise RuntimeError("Unknown environment: tpu_gtg {}, gpu_gtg {}".format(tpu_gtg, gpu_gtg))

if config["backend"] == "xla-tpu" and config["with_amp"]:
    raise RuntimeError("The value of with_amp should be False if backend is xla")


dist_configs = {'nproc_per_node': config["nproc_per_node"], "start_method": "fork"}  

def log_metrics(logger, epoch, elapsed, tag, metrics):
    metrics_output = "\n".join([f"\t{k}: {v}" for k, v in metrics.items()])
    logger.info(f"\nEpoch {epoch} - Evaluation time (seconds): {elapsed:.2f} - {tag} metrics:\n {metrics_output}")

with idist.Parallel(backend=config["backend"], **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=1)

2021-09-13 10:20:20,414 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-09-13 10:20:20,416 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-13 10:20:20,417 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fb8a7bd2710>' in 8 processes


local rank:  0
local rank:  1
local rank:  5
local rank:  2
local rank:  4
local rank:  7
local rank:  6
local rank:  3


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


2021-09-13 10:20:42,238 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 64, 'num_workers': 2, 'drop_last': True, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fb8a2131fd0>, 'pin_memory': False}
2021-09-13 10:20:42,263 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA
2021-09-13 10:20:42,286 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 64, 'num_workers': 2, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fb8a0465190>, 'pin_memory': False}
2021-09-13 10:20:42,302 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


2021-09-13 10:21:33,485 hkim-trainer INFO: Engine run starting with max_epochs=5.
2021-09-13 10:22:21,856 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-13 10:22:27,856 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:06
2021-09-13 10:22:27,892 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:06
2021-09-13 10:22:27,894 hkim-val_evaluator INFO: 
Epoch 1 - Evaluation time (seconds): 6.02 - validation evaluator metrics:
 	accuracy: 0.4528
	ce: 1.5992783203125
2021-09-13 10:22:27,903 hkim-trainer INFO: Epoch[1] Complete. Time taken: 00:00:54
2021-09-13 10:23:01,315 hkim-val_evaluator INFO: Engine run starting with max_epochs=1.
2021-09-13 10:23:04,701 hkim-val_evaluator INFO: Epoch[1] Complete. Time taken: 00:00:03
2021-09-13 10:23:04,713 hkim-val_evaluator INFO: Engine run complete. Time taken: 00:00:03
2021-09-13 10:23:04,723 hkim-val_evaluator INFO: 
Epoch 1 - Evaluation time (seconds): 3.38 - validation evaluator metrics:
 	accuracy:

 코드 실행 후에는 아래 그림에서와 같이 좌측 페인(pane)의 그리고 파일 아이콘을 클릭하면 보이는 리스트에서, batch_64_0.png와 같이 생성된 png 파일을 차례로 클릭한다.

<div align="center">
<img width=512 src="https://i.imgur.com/Y7owt0c.png"/>
</div> 

일례로 아래 그림은 프로세스 0번에서 auto_dataloader를 통해 준비된 미니배치에 포함된 이미지이다.

<div align="center">
<img width=512 src="https://i.imgur.com/Ng8S1Ei.png"/>
</div> 

그리고 다음 그림은 프로세스 1번에서 auto_dataloader를 통해 준비된 미니배치에 포함된 이미지이다.

<div align="center">
<img width=512 src="https://i.imgur.com/KPfwKqw.png"/>
</div> 

 이처럼 이그나이트에서 지원하는 auto_dataloader 함수에 의해 분산처리될 프로세스로 적절히 학습데이터가 배분되었음을 확인할 수 있다.
 


## License


---


Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  


---